# Make a movie of the kicker field

Using `Development/g-2/projects/20150529`

To open a window set up...  (for a `prof` build)

In [1]:
%%file iterm-kickerMovie-prof.scr
tell application "iTerm"
    activate
    set myterm to (make new terminal)
    tell myterm
        launch session "Default Session"
        tell the last session   
            set name to "project-20150529-prof"
            write text "source /cvmfs/oasis.opensciencegrid.org/gm2/prod/g-2/setup"
            write text "cd ~/Development/g-2/projects/20150529/try/"
            write text "source ../localProducts_gm2_v6_01_00_prof/setup"
            write text "setup ninja v1_5_3a"
            write text ". mrb s"
            write text "export PATH=$PATH:/Applications/paraview4.3.1.app/Contents/bin"
        end tell
    end tell
end tell

Overwriting iterm-kickerMovie-prof.scr


In [2]:
!osascript iterm-kickerMovie-prof.scr

I'm making a build on `gm2gpvm04`. Note that my version of the sources on my Mac is not from the latest develop. I do have `gm2dataproducts` checked out from the develop branch. That configuration of code fails to build on linux because there when I get the code for `gm2dataproducts` it's a newer `develop` branch! I don't want to upgrade all of my code yet, so I for `gm2dataproducts` checked out the branch that I have on my mac on Linux. E.g.

```bash
git checkout d79b0bfa86
```

Here's a script to log into gm2gpvm03

In [1]:
%%file iterm-kickerMovie-prof-gm2gpvm03.scr
tell application "iTerm"
    activate
    set myterm to (make new terminal)
    tell myterm
        launch session "Default Session"
        tell the last session   
            set name to "project-20150529-prof-gm2gpvm03"
            write text "ssh gm2gpvm03.fnal.gov"
            write text "screen"
            write text "cd /gm2/app/users/lyon/projects/20150529/kickerMovie"
            write text "source /grid/fermiapp/gm2/setup"
            write text "source ../localProducts_gm2_v6_01_00_prof/setup"
            write text "setup ninja v1_5_3a"
            write text ". mrb s"
            write text "export PATH=$PATH:/gm2/app/users/lyon/paraview/ParaView-4.3.1-Linux-64bit/bin"
            write text "source /grid/fermiapp/products/common/etc/setups.sh"
            write text "setup jobsub_client"
        end tell
    end tell
end tell

Overwriting iterm-kickerMovie-prof-gm2gpvm03.scr


In [2]:
!osascript iterm-kickerMovie-prof-gm2gpvm03.scr

I have an output area at `/pnfs/GM2/scratch/users/lyon/kickerMovie_20150709`

Here's the plan: I need a script to fill in parts of the `kickerMovie.fcl_TEMPLATE` for differnt arcs, different times and different file names.

## Prepare to run jobs

You can test jobs at `fermicloud168`. I've made a tar file on `gm2gpvm03:/gm2/app/users/lyon/projects/20150529/kickerMovieJob.tgz` . It contains the local products, paraview, and the fcl template.

I'm running the following:
* Based on gm2 v6_01_00 -q prof
* gm2ringsim feature/adamFieldDebug `2f7289fc0`
* gm2geom feature/nathan `61aa3f4e`
* gm2dataproducts (no branch - see above) `d79b0bfa8`

In `/gm2/app/users/lyon/g-2/projects/20150529` is all of the code and such.

In the top level, there is,
* kickerMovie - my interactive tests
* kickerMovieJob - Used to make the tar file
* kickerMovieSubmit - Has the submit script

## Submitting jobs

The submit script (see `kickerMovieSubmit/submit.sh`) submits the jobs. An example submit command is,

```bash
jobsub_submit -N 12 -G gm2 -M --OS=SL6 --tar_file_name=dropbox:///gm2/app/users/lyon/projects/20150529/kickerMovieJob.tgz --resource-provides=usage_model=DEDICATED,OPPORTUNISTIC --role=Analysis file:///gm2/app/users/lyon/projects/20150529/kickerMovieSubmit/workerNodeScript_allarcs.sh /pnfs/GM2/scratch/users/lyon/kickerMovie_20150709
```

Note the use of `dropbox://` for the tar file name. You must specify the driver script with `file://`. Anything after it are arguments to the script. It's all pretty convenient. 

I can use the `$PROCESS` ID to drive many of the choices, such as arc number and time step. 

I have two driver scripts:

The first one does different times for a particular arc

In [19]:
!cat workerNodeScript_arc3.sh

#!/bin/bash
#
#  This is the worker node script for kicker movie (and other magnetic field scans)
#
#  Be sure to submit with --tar_file_name of the tar file with the application
#

# Arguments are
# $1 = arc num
# $2 = output path

set -x

arcnum=$1
outpath=$2

# The fieldTime comes from the PROCESS number 75 + PROCESS * 5
fieldTime=$(expr 75 + $PROCESS \* 5)
echo $fieldTime

date
echo "Starting kickerMovie ${CLUSTER}.${PROCESS}  ${JOBSUBID}"

# We need IFDH
source /cvmfs/fermilab.opensciencegrid.org/products/common/etc/setups
setup ifdhc v1_6_2 -z /cvmfs/fermilab.opensciencegrid.org/products/common/db

echo "Environment for this job is " > job_env_${CLUSTER}.${PROCESS}.out
env >> job_env_${CLUSTER}.${PROCESS}.out

# Store the jobsub job id
if [ "${PROCESS}" == "0" ]; then
        echo ${JOBSUBJOBID} > ${JOBSUBJOBID}
        ifdh cp -D ${JOBSUBJOBID} $outpath
fi

# Setup g-2 stuff
source /cvmfs/oasis.opensciencegrid.org/gm2/prod/g-2/setup
setup gm2 v6_01_00 -q prof

# Now untar our st

This next one does an arc for a fixed time.

In [20]:
!cat workerNodeScript_allarcs.sh

#!/bin/bash
#
#  This is the worker node script for kicker movie (and other magnetic field scans)
#
#   Do all of the arcs (based on PROCESS number)
#
#  Be sure to submit with --tar_file_name of the tar file with the application
#

# Arguments are
# $1 = arc num
# $2 = output path

set -x

outpath=$1

arcnum=$PROCESS
fieldTime=800

date
echo "Starting kickerMovie ${CLUSTER}.${PROCESS}  ${JOBSUBID}"

# We need IFDH
source /cvmfs/fermilab.opensciencegrid.org/products/common/etc/setups
setup ifdhc v1_6_2 -z /cvmfs/fermilab.opensciencegrid.org/products/common/db

echo "Environment for this job is " > job_env_${CLUSTER}.${PROCESS}.out
env >> job_env_${CLUSTER}.${PROCESS}.out

# Store the jobsub job id
if [ "${PROCESS}" == "0" ]; then
        echo ${JOBSUBJOBID} > ${JOBSUBJOBID}
        ifdh cp -D ${JOBSUBJOBID} $outpath
fi

# Setup g-2 stuff
source /cvmfs/oasis.opensciencegrid.org/gm2/prod/g-2/setup
setup gm2 v6_01_00 -q prof

# Now untar our stuff
tar xvzf $INPUT_TAR_FILE

# Add our local

From the Seattle workshop, I think I need to take at least 1mm steps (grids seem to be in 0.5 mm steps). The grids tend to be cartesian, which is good. But may be in strange local coordinates. Would be nice to have polar coordinates instead of cartesian. 

I can have paraview calculate polar coordinates with a calculator. Then, by selecting points on the edges and labeling them with this data, we can divide up the space. Below is Arc 00 (in Geant numbering). 

In [1]:
!cp /Users/lyon/Development/g-2/projects/20150529/try/arcPositions/arc00.png .

![arc00](arc00.png)

A little hard to read. The tuples are $(\rho, \phi, y$) where $\rho = \sqrt{x^2+z^2}$ and $\phi = \tan^{-1}(z/x)$. $y$ is the vertical coordinate. This arc (00) has tracker chambers, and so its scallop is a little fatter than normal. From this, we see a range $7235 < \rho < 6575$, $0 < \phi < 0.524$ and $-80 < y < 80$ where linear units are mm.

We can split up phi a little better.

In [10]:
0.524*12

6.288

In [5]:
import math
2*math.pi

6.283185307179586

In [9]:
2*math.pi/12

0.5235987755982988

Looks like 0.524 for $\phi$ is a good step per arc. 

We need to look at arc11, since that one is bigger due to the yoke

Note that arc11 extends further in $\phi$ than the others. 

Inner radius of 6560 mm looks good

Outer radius of 7245 looks good too except for the inflector arc.
For the inflector arc, 9000mm looks good. 

If we use 0.524 radians spacing, then we'll have...

In [49]:
for a in range(12):
    x1 = 9000 * math.cos(a*0.524)
    z1 = 9000 * math.sin(a*0.524)
    x2 = 9000 * math.cos((a+1)*0.524)
    z2 = 9000 * math.sin((a+1)*0.524)
    print "arc {}, x1={}, z1={}; x2={}, z2={}".format(a, x1, z1, x2, z2)

arc 0, x1=9000.0, z1=0.0; x2=7792.42249694, z2=4503.12687243
arc 1, x1=7792.42249694, z1=4503.12687243; x2=4493.7440824, z2=7797.83714384
arc 2, x1=4493.7440824, z1=7797.83714384; x2=-10.8330562301, z2=8999.99348027
arc 3, x1=-10.8330562301, z1=8999.99348027; x2=-4512.5031382, z2=7786.99656015
arc 4, x1=-4512.5031382, z1=7786.99656015; x2=-7803.24049301, z2=4484.35478171
arc 5, x1=-7803.24049301, z1=4484.35478171; x2=-8999.97392109, z2=-21.6660967649
arc 6, x1=-8999.97392109, z1=-21.6660967649; x2=-7781.55934133, z2=-4521.87286612
arc 7, x1=-7781.55934133, z1=-4521.87286612; x2=-4474.95898395, z2=-7808.63253662
arc 8, x1=-4474.95898395, z1=-7808.63253662; x2=32.4991059092, z2=-8999.94132248
arc 9, x1=32.4991059092, z1=-8999.94132248; x2=4531.23604262, z2=-7776.11084837
arc 10, x1=4531.23604262, z1=-7776.11084837; x2=7814.01326686, z2=-4465.55670274
arc 11, x1=7814.01326686, z1=-4465.55670274; x2=8999.8956845, z2=43.3320679679


The above works great!

Now, how many points do we want to do?

For normal arcs,

In [29]:
def nsteps(pmin, pmax, stepSize):
    return (pmax-pmin)/stepSize

In [30]:
ymin=-80; ymax=80 ; nYsteps=nsteps(ymin, ymax, 1.0) ; nYsteps

160.0

In [31]:
rmin=6560; rmax=7245; nRsteps = nsteps(rmin, rmax, 1.0) ; nRsteps

685.0

s = $r \phi$

In [32]:
phiSpacing = 0.75/7245; phiSpacing

0.00010351966873706004

In [33]:
phiSpacing = math.floor(phiSpacing*1000000)/1000000.0 ; phiSpacing

0.000103

In [34]:
0.524/phiSpacing

5087.378640776699

In [35]:
2*math.pi/phiSpacing

61001.79909883094

Make the phi steps an even 50000

In [36]:
phiSteps=50004

In [37]:
dphi = (2*math.pi)/phiSteps ; dphi

0.00012565365385128362

How many steps per arc?

In [38]:
phiSteps/12.

4167.0

In [46]:
0.524 / (phiSteps/12.0)

0.00012574994000479962

In [39]:
phiSteps*dphi 

6.283185307179586

In [40]:
6575*dphi

0.8261727740721898

In [41]:
9000*dphi

1.1308828846615526

So, how many points total will we have?

In [44]:
totalPoints = (nYsteps * nRsteps * phiSteps)/1e9 ; totalPoints  # IN Billions

5.4804384

In [45]:
totalPoints/12*1e3  # IN Millions - per arc

456.7032

Need animation of arcs 2 and 3